# Exploring the UTx000 Extension Beacon Data
(Known as BPEACE2 in the [GH repo](https://github.com/intelligent-environments-lab/utx000))

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Evening IEQ Measurements
Now that we have parsed the beacon data into time periods corresponding to when participants were asleep, it would be interesting to analyze what concentrations are being experienced during the evening.

## Package Import

In [2]:
import os
import sys
sys.path.append('../')

from src.features import build_features
from src.visualization import visualize
from src.reports import make_report

import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# Data Import
We are only concerned with the beacon data when participants are asleep since that is the only time we can be sure participants are home in their bedrooms.

In [5]:
beacon = pd.read_csv('../data/processed/bpeace2-beacon-fb_ema_and_gps_restricted.csv',
                    index_col=0,parse_dates=[0,-1,-2],infer_datetime_format=True)
beacon.head()

,Lat,Long,Alt,Accuracy,TVOC,eCO2,Lux,Visible,Infrared,NO2,...,PM_C_2p5,PM_C_4,PM_C_10,z,Beacon,Beiwe,Fitbit,REDCap,start_time,end_time
2020-08-10 05:10:00,30.58838,-97.42591,113.98145,5.29684,185.50,750.25,2.04,1.0,0.0,288.95,...,6.748117,10.714056,11.507244,1.441551,21,lkkjddam,25,12,2020-08-10 04:42:30,2020-08-10 12:35:30
2020-08-10 05:15:00,NaN,NaN,NaN,NaN,166.08,706.48,2.04,1.0,0.0,277.12,...,6.515844,10.288899,11.043510,1.468756,21,lkkjddam,25,12,2020-08-10 04:42:30,2020-08-10 12:35:30
2020-08-10 05:20:00,NaN,NaN,NaN,NaN,151.40,666.48,2.04,1.0,0.0,277.44,...,6.215933,9.795078,10.510907,1.492578,21,lkkjddam,25,12,2020-08-10 04:42:30,2020-08-10 12:35:30
2020-08-10 05:25:00,NaN,NaN,NaN,NaN,158.84,686.92,2.04,1.0,0.0,282.44,...,6.486098,10.357869,11.132223,1.457762,21,lkkjddam,25,12,2020-08-10 04:42:30,2020-08-10 12:35:30
2020-08-10 05:30:00,NaN,NaN,NaN,NaN,146.15,669.20,2.04,1.0,0.0,272.80,...,6.721101,10.739605,11.543306,1.430890,21,lkkjddam,25,12,2020-08-10 04:42:30,2020-08-10 12:35:30


# Finding Nights with Constant CO2 Concentration
We need to find nights when the CO2 concentration is constant for at least an hour.

In [18]:
good_nights = pd.DataFrame()
for pt in beacon['Beiwe'].unique():
    beacon_pt = beacon[beacon['Beiwe'] == pt]
    for start, end in zip(beacon_pt['start_time'],beacon_pt['end_time']):
        if (end - start).total_seconds()/3600 > 5: # sleeping longer than 5 hours
            beacon_pt_night = beacon_pt[start:end] # iaq data during sleep
            std = np.std(beacon_pt_night)
            diffs = [0]
            for i in range(len(beacon_pt_night)-1):
                diff = abs(beacon_pt_night['CO2'][i+1] - beacon_pt_night['CO2'][i])
                diffs.append(diff)
            beacon_pt_night['differences'] = diffs
            no_change = beacon_pt_night[beacon_pt_night['differences'] < std]
            num_mins = []
            for i in range(len(no_change)):
                dts = []
                

ValueError: Can only compare identically-labeled Series objects

In [17]:
beacon_pt_night

NameError: name 'beacon_pt_night' is not defined